In [1]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [9]:
import json
import requests

Key = '06e95084cb5d3426c57b51987827eb7d'
url = "https://developers.zomato.com/api/v2.1/categories"

if __name__ == '__main__':
    r = requests.get(url, headers={'user-key': Key})
    if r.ok:
        data = r.json()
        print(data)

{'categories': [{'categories': {'id': 1, 'name': 'Delivery'}}, {'categories': {'id': 2, 'name': 'Dine-out'}}, {'categories': {'id': 3, 'name': 'Nightlife'}}, {'categories': {'id': 4, 'name': 'Catching-up'}}, {'categories': {'id': 5, 'name': 'Takeaway'}}, {'categories': {'id': 6, 'name': 'Cafes'}}, {'categories': {'id': 7, 'name': 'Daily Menus'}}, {'categories': {'id': 8, 'name': 'Breakfast'}}, {'categories': {'id': 9, 'name': 'Lunch'}}, {'categories': {'id': 10, 'name': 'Dinner'}}, {'categories': {'id': 11, 'name': 'Pubs & Bars'}}, {'categories': {'id': 13, 'name': 'Pocket Friendly Delivery'}}, {'categories': {'id': 14, 'name': 'Clubs & Lounges'}}]}


In [3]:
# Connect to BeautifulSoup and define the parser as HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find all apts that fit the specified criteria and # of apts
apts = soup.find_all('p', attrs={'class':'result-info'})
print(len(apts))

27


##  Parse through Craigslist results

In [7]:
# results are returned as an iterable list
results = soup.find_all('li', class_="result-row")
list_results = []


# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        # Identify and return price of listing
        price = result.a.span.text.strip('$')
        # Identify number of rooms
        n_rooms = result.find('span', class_ = 'housing').text.split()
        # Identify and return link to listing
        link = result.a['href']
        # Identify posting date
        post_time = result.find('time')['datetime']
        post_time = pd.to_datetime(post_time)
        # Identify neighborhood
        neighborhood = result.find('span', class_= 'result-hood').text
        
        list_results.append({'#bedrooms': n_rooms, '$price': price,
                        'Post_Title': title, 'URL': link, 'Post_Time': post_time, 'Neighborhood': neighborhood})
        
        # Print results only if title, price, and link are available
        if (title and price and link):
            print('-------------')
            print(title)
            print(price)
            print(n_rooms)
            print(link)
            print(post_time)
            print(neighborhood)
    except AttributeError as e:
        print(e)
        

-------------
2 bedroom with hardwood and large rooms near Polk
3250
['2br', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-2-bedroom-with-hardwood/6818165978.html
2019-02-12 21:13:00
 (nob hill)
-------------
Prime North Beach, Union @ Stockton 2BR/1BA, good size unit for rent..
3450
['2br', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-prime-north-beach-union/6813029028.html
2019-02-12 17:31:00
 (north beach / telegraph hill)
-------------
Bright and Spacious - Newly renovated 1Br w/Den
3095
['2br', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-bright-and-spacious-newly/6816304843.html
2019-02-12 17:08:00
 (lower nob hill)
-------------
Hyde/Clay Sts 2 bdrm, liv rm, xtra bonus rm, carpet, lar kit, grt loc
3100
['2br', '-', '850ft2', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-hyde-clay-sts-2-bdrm-liv/6817385660.html
2019-02-12 00:04:00
 (nob hill)
'NoneType' object has no attribute 'text'
-------------
Bright and Spacious - Newly renov

## Create DataFrame from results

In [8]:
# Create DataFrame
apt_df = pd.DataFrame(list_results)

# Clean data and remove text for room/ 
bedrooms = apt_df['#bedrooms']
rooms = []

for room in bedrooms:
    rooms.append(room[0].strip('br'))

apt_df['#rooms'] = rooms
apt_df['Post_Time'] = apt_df['Post_Time'].dt.date

apt_df

,#bedrooms,$price,Neighborhood,Post_Time,Post_Title,URL,#rooms
0,"[2br, -]",3250,(nob hill),2019-02-12,2 bedroom with hardwood and large rooms near Polk,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
1,"[2br, -]",3450,(north beach / telegraph hill),2019-02-12,"Prime North Beach, Union @ Stockton 2BR/1BA, g...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
2,"[2br, -]",3095,(lower nob hill),2019-02-12,Bright and Spacious - Newly renovated 1Br w/Den,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
3,"[2br, -, 850ft2, -]",3100,(nob hill),2019-02-12,"Hyde/Clay Sts 2 bdrm, liv rm, xtra bonus rm, c...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
4,"[2br, -]",3095,(lower nob hill),2019-02-11,Bright and Spacious - Newly renovated 1Br w/Den,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
5,"[2br, -, 750ft2, -]",3495,(nob hill),2019-02-11,New Two Bedroom in Nob Hill,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
6,"[2br, -, 515ft2, -]",2988,(nob hill),2019-02-11,2 bedroom ​freshly remodeled. Clay & Mason,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
7,"[2br, -]",2950,(nob hill),2019-02-11,2 Bedroom 1 Bathroom Recently Renovated Apt Av...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
8,"[2br, -]",3500,(north beach / telegraph hill),2019-02-11,"Charming 2/1, HW, laundry, great location!",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
9,"[2br, -]",3400,(nob hill),2019-02-10,2 BR Available For Rent- $3400,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2


In [9]:
# Clean data and remove text from #bedrooms and create new column for sqft 

# Split before and after the dash & remove the 'ft2' text
apt_df['#bedrooms'] = apt_df['#bedrooms'].astype(str)
apt_df['#bedrooms'] = apt_df['#bedrooms'].str.split(',').str[2]
apt_df['#bedrooms'].fillna("", inplace=True)
apt_df['#bedrooms'] = apt_df['#bedrooms'].str.split('ft2') 

###############################################################

# Loop through the new bedroom values and remove remaining text that will prevent converting to int
bedrooms = apt_df['#bedrooms']
rooms = []

for room in bedrooms:
    rooms.append(room[0].strip(','))

apt_df['sqft'] = rooms
apt_df['sqft'] = apt_df['sqft'].str.replace(r"'", '')
apt_df['sqft'] = apt_df['sqft'].astype(str)
apt_df['sqft'] = apt_df['sqft'].str.strip()

##############################################################

# Convert values to int and if NAN make = 0
sqfts = apt_df['sqft']
number = []

for sqft in sqfts:
    number.append(sqft)

newsqft = pd.to_numeric(number, errors='ignore')
apt_df['sqft'] = newsqft
apt_df['sqft'].fillna("0", inplace=True)
apt_df['sqft'] = apt_df['sqft'].astype(int)

##############################################################

# Convert price and # rooms to int in order to graph data
apt_df['$price'] = apt_df['$price'].astype(int)
apt_df['#rooms'] = apt_df['#rooms'].astype(int)


##############################################################

# Clean up the Neighborhood name
apt_df['Hood'] = apt_df['Neighborhood'].str.extract(r"\((.*?)\)", expand=False)
apt_df['Hood'] = apt_df['Hood'].astype(str)
apt_df['Hood'] = apt_df['Hood'].str.strip()

neighborhoods = apt_df["Hood"].unique()
new_hoods = ['North Beach', 'Lower Nob Hill', 'Nob Hill', 'Russian Hill', 'Lower Pac Hts', 'Pac Heights']
my_dict = dict(zip(neighborhoods, new_hoods))

apt_df['Hood'].replace(my_dict, inplace=True)


apt_df

,#bedrooms,$price,Neighborhood,Post_Time,Post_Title,URL,#rooms,sqft,Hood
0,[],3250,(nob hill),2019-02-12,2 bedroom with hardwood and large rooms near Polk,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,North Beach
1,[],3450,(north beach / telegraph hill),2019-02-12,"Prime North Beach, Union @ Stockton 2BR/1BA, g...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,Lower Nob Hill
2,[],3095,(lower nob hill),2019-02-12,Bright and Spacious - Newly renovated 1Br w/Den,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,Nob Hill
3,"[ '850, ']",3100,(nob hill),2019-02-12,"Hyde/Clay Sts 2 bdrm, liv rm, xtra bonus rm, c...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,850,North Beach
4,[],3095,(lower nob hill),2019-02-11,Bright and Spacious - Newly renovated 1Br w/Den,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,Nob Hill
5,"[ '750, ']",3495,(nob hill),2019-02-11,New Two Bedroom in Nob Hill,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,750,North Beach
6,"[ '515, ']",2988,(nob hill),2019-02-11,2 bedroom ​freshly remodeled. Clay & Mason,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,515,North Beach
7,[],2950,(nob hill),2019-02-11,2 Bedroom 1 Bathroom Recently Renovated Apt Av...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,North Beach
8,[],3500,(north beach / telegraph hill),2019-02-11,"Charming 2/1, HW, laundry, great location!",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,Lower Nob Hill
9,[],3400,(nob hill),2019-02-10,2 BR Available For Rent- $3400,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,0,North Beach


In [10]:
# New DataFrame to build graphs in
apts_data = apt_df[['#rooms', 'sqft', '$price', 'Hood', 'Post_Time', 'Post_Title', 'URL']]

apts_data.dtypes

#rooms         int32
sqft           int32
$price         int32
Hood          object
Post_Time     object
Post_Title    object
URL           object
dtype: object

# Send data to SlackBot

In [35]:
import os
from slackclient import SlackClient
import pprint
from collections import Counter

In [ ]:
slack_data = apts_data.to_dict('records')
str(slack_data)

In [ ]:
# Prints the nicely formatted dictionary
pprint.pprint(slack_data)

# Sets 'pretty_dict_str' to 
pretty_dict_str = pprint.pformat(slack_data)

In [ ]:
SLACK_TOKEN = "ENTER_SLACK_API"
SLACK_CHANNEL = "ENTER_SLACK_CHANNEL"

sc = SlackClient(SLACK_TOKEN)

sc.api_call(
    "chat.postMessage", channel=SLACK_CHANNEL, text=pretty_dict_str,
    username='pybot', icon_emoji=':robot_face:'
)